# Wrangling Twitter data
In this notebook, we are going to start out with the raw Twitter data that was captured using the Tweepy tool on which you can read more about in [this notebook](tweepy_bot.ipynb). The data captured was restricted to tweets from Alberta from December 18 2018 to December 27 2018. 

In order to prep the data for analysis, we will "wrangle" the data to clean it and structure it into a more useful format. 

## Goal: 
Learn some common aspects of data wrangling and convert the data set into a format more practical for further data exploration and analysis. 


## Data wrangling introduction
[Data wrangling](https://en.wikipedia.org/wiki/Data_wrangling) "is the process of transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics" (Wikipedia).

The purpose of data wrangling notebook is to transform complex data into a format that is easier to work with in order to focus on the analysis of data, rather than the wrangling in the next notebooks. 

#### Common data wrangling steps:
 - **Discovering**  - understanding the data
 - **Structuring** -  organizing the data
     - Naming or renaming variables
     - Changing the type of variables
     - Creating new variables through functions of existing of variables
     - Conditional processing of variables (i.e the values of new variable is based on the values of existing variables) 
 - **Cleaning** 
     - Redesign the data into a usable/functional format and correct/remove any bad data
     - Imputing missing values in a variables
 - **Publishing**
    - Selecting columns from input dataset to output dataset


## Load python modules

There are many packages available in Python that provide a variety of functions for data science. Here we load the libraries needed for this notebook. 

One of the most common libraries used for data science is [pandas](https://pandas.pydata.org/). There are good tutorials available on pandas and worthwhile spending more time with in order to become more comfortable with data science in python. 

In [ ]:
try:
    import urllib.request
except ImportError:
    !pip install  --user urllib
    import urllib.request

try:
    import ast
except ImportError:
    !pip install  --user ast
    import ast
    
try:
    import pandas as pd
except ImportError:
    !pip install  --user pandas
    import pandas as pd

try:    
    import matplotlib.pyplot as plt
except ImportError:
    !pip install --user matplotlib
    import matplotlib.pyplot as plt

%matplotlib inline

## Step 1. Taking a look at the raw data


#### Reading the Twitter data  and displaying the first 5 rows

The raw data(`tweets.txt`) is stored in the same directory, we will read it into a pandas DataFrame. A **DataFrame** is a commonly encountered way to work with data in data science. It can be thought of as a 2D data structure that makes it easy to work with the data at hand. 

In [ ]:
file_name="alberta_tweets.txt"
tweets = []
with open(file_name, 'r') as f:
    for lines in f.readlines():
        tweets.append(ast.literal_eval(lines))
frame  = pd.DataFrame(tweets)
frame.head(3) # head is a convenient way for showing the first few rows of a dataframe

#### Understanding the raw data

Let's take a look at some basic features of the raw data. For example, it's always a good idea to look at the columnns and the size of the data in question. 

In [ ]:
frame.columns

In [ ]:
frame.shape

Taking a more complete look at the data, here the size of the DataFrame are printed, along with the: 
* column name
* column type
* sample value in the column

In [ ]:
print("Number of rows/columns:",frame.shape, "\n")
for column in frame.columns:
    if column=="user_mentions":
        rownum=1
    else :
        rownum=2
    print("Column name: {colname} \nColumn type: {coltype} \nColumn data(example): {coldata} \n\n"
          .format(colname=column, coltype=type(frame[column].iloc[rownum]),coldata=frame[column].iloc[rownum]))

There are several interesting features there, such as latitude and longitude of the tweets (`coordinates` column), and most of the columns make sense and are pretty intuitive. Two columns that are a bit confusing are `text` and `extended_tweet` as it looks like they may contain the same content. We should figure out if they are actually different. 

### Indexing in DataFrames

DataFrames are convenient for working with data and are popular both in Python and R. There are a few different ways for accessing data inside a DataFrame. We'll cover a couple of aspects here, but you may also want to read more in the  pandas documentation or cheat sheet such as the one handed out [here](https://www.dataquest.io/blog/large_files/pandas-cheat-sheet.pdf). 

In [ ]:
pd.set_option('display.max_row', 10) #Ensure only 10 rows are displayed to save space
pd.set_option('display.max_colwidth', -1) #Ensure output is not truncated by width

#Selecting a column
frame[["text"]] #Using two brackets displays the column as a dataframe

In [ ]:
#Selecting two columns 
frame[["text", "extended_tweet"]]

In [ ]:
#Selecting row subsets
selecttwo = frame[["text", "extended_tweet"]].iloc[[2,5]] #iloc allows for selection by integers
selectrange = frame[["text", "extended_tweet"]].iloc[2:5]

display(selecttwo)
display(selectrange)

In [ ]:
#Select a certain row, all columns
frame.iloc[3,:]

#### Exercise: Use another method for displaying the same data as above (i.e. all columns of row 3) 

The answer should look as above or capture the same data

In [ ]:
###### Enter your code below



#### Investigating `text` and `extended_tweet` graphically

Another way to investigate the difference between `text` and `extended_tweet` is to plot the length distribution for both columns. To do this, we use the [matplotlib library](https://matplotlib.org/) (another very common library used in data science). 

In [ ]:
#Set up a plot with subplots 
plt.subplot(1, 2, 1) # (rows, columns, active plot)
#Create a histogram, which counts the number of tweets of each length
plt.hist(frame["text"].apply(len).values, color = 'green', edgecolor = 'black')
#Label the plot
plt.title('text')
plt.xlabel('Tweet length')
plt.ylabel('Number of tweets')

# Do the same thing for the second subplot
plt.subplot(1, 2, 2)
plt.hist(frame["extended_tweet"].apply(len).values, color = 'blue', edgecolor = 'black')
plt.title('extended_tweet')
plt.xlabel('Tweet length')

Looks like `extended_tweet` has much longer tweets, up to 800 characters in length, whereas there are no tweets longer than 150 characters in `text`. 

What does the longest tweet look like? To do this, we can easily pull the longest tweet from the DataFrame. 


In [ ]:
print("Maximum length for extended_tweet column: ",max(frame["extended_tweet"].apply(len)), "\n")

print("This is the index of the longest tweet:")
print([frame["extended_tweet"].apply(len).idxmax(axis=1)], "\n")

#### Exercise: Using the index you determined above, add your code to display what the longest tweet looks like in the `text` and `extended_tweet` columns in the cell below.

In [ ]:
print("Tweet as seen in the **extended_text** column:\n")
###### Enter your code in place of None below
extended = None
######

print(str(extended) + "\n")

print("Tweet as seen in the **text** column:\n")
###### Enter your code in place of None below
text_tweet = None
######
print(text_tweet)

The answer should show that the `text` column is the truncated version of `extended_tweet` column.

#### So we have the following list of columns:
 - **coordinates** - user coordinates, can be None or Python type Dict(needs to be split to two columns - longitude and latitude)
 - **created_at**  - creation date, Python type String (needs to be converted to type Date + timezone needs to be changed from UTC to MST)
 - **extended_tweet** - tweet text, Python type String
 - **hashtags** - hashtags, Python type List of Dicts(needs to be converted to String)
 - **lang** - language , Python type String
 - **name** - twitter account name, Python type String
 - **screen_name** - screen name, Python type String
 - **text** - truncated tweet text, Python type String
 - **user_location** - location, Python type String
 - **user_mentions** - user mentions, Python type List of Dicts(needs to be converted to String)

## Step 2. (Re-) structuring  the data

One aspect to always pay close attention to is the data type of each column. Overlooking data types and ending up with unexpected types often leads to issues further downstream in the analysis. Here we will go through and change some of the data types to make sure they are what we want them to be. 

As part of the process we will print the original data example and converted data example. To get specific row/column from pandas dataframe we will use following:
> datafarme_name["column_name"].iloc[row_number]

#### Convert column `created_at`  from String to Date and change timezone from UTC to MST

In [ ]:
print(frame["created_at"].iloc[2]) ## Print the original data

frame["created_at_date"]= pd.to_datetime(frame["created_at"]) ## convert to timestamp
frame["created_at_date"]=frame["created_at_date"].dt.tz_localize('UTC').dt.tz_convert('MST') ## fix timezone

print(frame["created_at_date"].iloc[2]) ## Print the transformed data

#### Convert column `coordinates` from dictionary to two columns `latitude` and `longitude`

In [ ]:
print(frame["coordinates"].head(), "\n") ## Print the original data

def convertCoordToList(v):  ## helper function
    if v is None:
        v=[]
    else:
        v= v['coordinates']
    return v

frame['coordinates_list']= frame['coordinates'].map(convertCoordToList) #Convert coordinates from Dict to List

#Split coordinates_list column into two: latitude and longitude
frame[['latitude','longitude']] = pd.DataFrame(frame['coordinates_list'].values.tolist())

print(frame[['longitude','latitude']].head()) ## Print the transformed data

#### Convert column `hashtags`  from dictionary to strings
If there were no hashtags in a cell, we will fill that cell in our dataframe with a single space `' '`. 

In [ ]:
print(frame["hashtags"].iloc[1]) ## Print the original data

def convertHashToString(v): ## helper function
    if v == []:
        v=' '
    else:
        v=' '.join([d['text'] for d in v])
    return v

frame['hashtags_string']= frame['hashtags'].map(convertHashToString) ## Do the conversion

print("\n",frame["hashtags_string"].iloc[2]) ## Print the transformed data

#### Convert column `user_mentions` from dictionary to string

We don't want the `user_mentions` column to be in a dictionary format as it is easier to work with as a string. Let's convert it.

The following is a sample of what the data looks like and what data type it is.

In [ ]:
print(frame["user_mentions"].iloc[2]) ## Print the original data + type
print(type(frame["user_mentions"].iloc[2]))

**Exercise:** Using the `hashtags` conversion above as an example, complete the following cell in order to convert the `user_mentions` column to strings. 

In [ ]:
def convertUserToString(v):
    if v == []:
        v=' '
    else:
        v=' '.join([d['screen_name'] for d in v])
    return v

###### Enter your code below


######


#### Check your answer
Only screen_name's should be extracted from the distionary and data type should be str

In [ ]:
print(frame["user_string"].iloc[2]) ## Print the transformed data + type
print(type(frame["user_string"].iloc[2]))


## Step 3. Cleaning the data

Some data cleanup was done in step 2. Let's take a closer look at the `extended_tweet` column, since we will use it in further during our data analysis. 

First, let's look at what some of the tweets look like. We'll set an output parameter and then access the first few tweets. 

In [ ]:
pd.set_option('max_colwidth', 700) ## Setting col_widht wider - so we can see the entire tweet
frame['extended_tweet'].head(10)

####  Remove URLS 
There are lots of links in this column that might make our future data analysis messy and likely do not have any meaning. Let's remove them together with any links starting with www.  

We will accomplish this using regular expressions (Note that **\S+** means one or more non-whitespace character)

In [ ]:
frame['extended_tweet']=frame['extended_tweet'].str.replace('http\S+|www.\S+', '', case=False)

#### Exercise: remove '\n' (new line symbol) from 'extended_tweet' column

Another item that might interfere with our analysis are new line characters. Let's remove these as well in the cell below. 

In [ ]:
##### Enter your code below



######


#### Check your answer
Print the first 10 tweets and see if \n is removed

In [ ]:
frame['extended_tweet'].head(10)

#### Remove user mentions and hashtags from the 'extended_tweet' column and save it as 'extended_tweet_cleaned' column
There are separate columns for user mentions and hashtags. We might need to analyse tweet text without any hashtags/user mentions. However, we might also want to do some analysis on the tweets including the mentions and hashtags. As such, tweets with these items removed will be stored in a separate column. 

In [ ]:
frame['extended_tweet_cleaned']=frame['extended_tweet'].str.replace('[@#]\w+', ' ',case=False)
frame['extended_tweet_cleaned'].head(5)

We will keep emoji, punctuation,  numbers and special characters for now. They might be useful for the analysis. We can always do more cleaning in the later notebooks

#### Let's take a look at the language column

In [ ]:
frame["lang"].unique()

Column lang has only one value en so it can be safely ignored



## Step 6. Publishing - saving data for future work

In order to work with this data further, let's save a new DataFrame. 

We'll simply select the desired columns from the working dataset into the output dataset and take a look at the output. Does the output match what we expected? How do the `extended_tweet` and `extended_tweet_clean` columns compare? 

Further below, we'll save the data to a csv file. 

In [ ]:
frame_cleaned=frame[['created_at_date','hashtags_string','user_string','user_location',
                     'longitude','latitude','name', 'screen_name','extended_tweet','extended_tweet_cleaned']]
frame_cleaned.head()

In [ ]:
frame_cleaned.to_csv('alberta_tweets_cleaned.csv',index=False)

##  Conclusion

We've taken a preliminary look at some of the raw Twitter data that's been collected in order to further work with it. This included restructuring and reorganizing it and cleaning up the columns that we anticipate needing the most for our analysis.

Take-aways: 
* We've started working with some very popular libraries for doing data science in Python: pandas and matplotlib
* Examining, wrangling, and cleaning your data are important work that will take up large portions of time in the data science workflow
* This is an iterative process. We took a stab at what we need to clean up and refine now, but may very well find things we've missed as we dig deeper into the data. 

Next, we will get into data exploration further. 